**Dependencies**

In [1]:
from langchain_ollama import ChatOllama
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent
from sql import run_sql_workflow
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_core.messages import SystemMessage, HumanMessage, ToolMessage
from typing import Annotated
from langchain_core.tools import tool, BaseTool, InjectedToolCallId
from langchain_core.messages import ToolMessage
from langgraph.types import Command
from langgraph.prebuilt import InjectedState

In [2]:
local_model = ChatOllama(model="qwen3:30b-a3b", temperature=0.1)

import re

def remove_think_blocks(text: str) -> str:
    return re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL).strip()
    
def create_custom_handoff_tool(*, agent_name: str, name: str | None, description: str | None) -> BaseTool:
    @tool(name, description=description)
    def handoff_to_agent(
        # you can add additional tool call arguments for the LLM to populate
        # for example, you can ask the LLM to populate a task description for the next agent
        task_description: Annotated[str, "Detailed description of what the next agent should do, this should always be a string, if more than one line, use triple quotes."],
        # you can inject the state of the agent that is calling the tool
        state: Annotated[dict, InjectedState],
        tool_call_id: Annotated[str, InjectedToolCallId],
    ):
        tool_message = ToolMessage(
            content=f"Successfully transferred to {agent_name}",
            name=name,
            tool_call_id=tool_call_id,
        )
        messages = state["messages"]
        prompt= ""
        for m in messages[::-1]:
            if m.name == "prompt_agent" and "Transferring" not in m.content:
                prompt = m.content
                prompt= remove_think_blocks(prompt)
                break
        if prompt:
            print(f"Prompt handoff: {prompt}")
            task_description = {"original_question":task_description, "prompt": prompt}
        return Command(
            goto=agent_name,
            graph=Command.PARENT,
            # NOTE: this is a state update that will be applied to the swarm multi-agent graph (i.e., the PARENT graph)
            update={
                "messages": messages + [tool_message],
                "active_agent": agent_name,
                # optionally pass the task description to the next agent
                # NOTE: individual agents would need to have `task_description` in their state schema
                # and would need to implement logic for how to consume it
                "task_description": task_description,
            },
        )

    return handoff_to_agent

In [3]:
@tool("get_cases_schema", description="Returns the schema of the cases table")
def get_cases_schema() -> str:
    return """
    === CASES TABLE ===

    You are a SQL assistant with access to the `cases` table. Below is the structure and brief description of each column.

    Structure:
        id                     VARCHAR       -- Unique identifier for each case
        order_date             TIMESTAMP_NS  -- Date the order was placed
        employee_id            VARCHAR       -- ID of the employee handling the case
        branch                 VARCHAR       -- Branch responsible for the case
        supplier               VARCHAR       -- Supplier associated with the case
        avg_time               DOUBLE        -- Average time to complete the case
        estimated_delivery     TIMESTAMP_NS  -- Estimated delivery date
        delivery               TIMESTAMP_NS  -- Actual delivery date
        on_time                BOOLEAN       -- Whether the delivery was on time
        in_full                BOOLEAN       -- Whether the delivery was complete
        number_of_items        INTEGER       -- Total items in the case
        ft_items               INTEGER       -- Fast-track items
        total_price            DOUBLE        -- Total price of the case
        total_activities       INTEGER       -- Total activities in the process
        rework_activities      INTEGER       -- Count of rework activities
        automatic_activities   INTEGER       -- Count of automated activities

    Instructions:
    - Use standard SQL (SELECT, WHERE, GROUP BY, etc.).
    - Use `order_date` for filtering by time.
    - Use aggregations (COUNT, SUM, AVG) for metrics.
    """


@tool("get_activities_schema", description="Returns the schema of the activities table")
def get_activities_schema() -> str:
    return """
    === ACTIVITIES TABLE ===

    You are a SQL assistant with access to the `activities` table. Below is the schema with brief descriptions of each column.

    Structure:
        id                       INTEGER     -- Unique ID for each activity
        timestamp                TIMESTAMP   -- Time the activity occurred
        name                     VARCHAR     -- Name/type of activity
        tpt                      DOUBLE      -- Time per task
        user                     VARCHAR     -- User who performed the activity
        user_type                VARCHAR     -- Type of user (e.g., Human, Bot)
        automatic                BOOLEAN     -- Whether the activity was automated
        rework                   BOOLEAN     -- Whether the activity was a rework
        case_index               INTEGER     -- Index of the activity in the case
        case_id                  VARCHAR     -- ID of the related case

    Case metadata (prefixed with `case_`):
        case_order_date          TIMESTAMP   -- Order date of the case
        case_employee_id         VARCHAR     -- Employee responsible
        case_branch              VARCHAR     -- Responsible branch
        case_supplier            VARCHAR     -- Supplier involved
        case_avg_time            DOUBLE      -- Average processing time
        case_estimated_delivery  TIMESTAMP   -- Expected delivery date
        case_delivery            TIMESTAMP   -- Actual delivery date
        case_on_time             BOOLEAN     -- Whether the case was on time
        case_in_full             BOOLEAN     -- Whether the delivery was complete
        case_number_of_items     INTEGER     -- Number of items in the case
        case_ft_items            INTEGER     -- Fast-track items
        case_total_price         DOUBLE      -- Total case price

    Instructions:
    - Standard SQL syntax is allowed (WHERE, GROUP BY, etc.).
    - Use `automatic` and `rework` for process analysis.
    - Use `timestamp`, `name`, or `user_type` for activity-based queries.
    - You may aggregate case-related columns, but do not reference other tables.
    """

@tool("get_variants_schema", description="Returns the schema of the variants table")
def get_variants_schema() -> str:
    return """
    === VARIANTS TABLE ===

    You are a SQL assistant with access to the `variants` table. Below is the schema with short descriptions for each column.

    Structure:
        id              BIGINT       -- Unique ID for each variant
        activities      VARCHAR[]    -- Ordered list of activity names in this variant
        cases           VARCHAR[]    -- Array of case IDs following this variant
        number_cases    BIGINT       -- Number of cases following this variant
        percentage      DOUBLE       -- Share of total cases for this variant
        avg_time        DOUBLE       -- Average processing time for this variant

    Instructions:
    - Each row represents a unique process path ("variant") followed by one or more cases.
    - Use `number_cases`, `percentage`, or `avg_time` to rank, filter, or compare variants.
    - Use array functions (e.g., `ANY`, `UNNEST`, `array_length`) to inspect activities or case IDs.
    - Standard SQL syntax allowed (WHERE, ORDER BY, LIMIT, etc.).
    - Deviations are variants that differ from the most common one (highest `number_cases`).
    - Do not reference other tables.
    """


prompt_agent = create_react_agent(
    model=local_model,
    tools=[get_cases_schema, get_activities_schema, get_variants_schema],
    name="prompt_agent",
    prompt="""/no_think
You are a prompt formatting agent. Your task is to create a detailed prompt for the next agent based on a natural language query. The prompt should include relevant table schemas and instructions for querying those tables. Follow these steps in the exact order:

### IMPORTANT
- **ALWAYS CALL THE RELEVANT TOOLS TO FETCH THE SCHEMAS**
- **DO NOT GUESS THE SCHEMAS OF TABLES**

## Step-by-Step Instructions

1. **Identify Relevant Tables**  
   Analyze the user question and determine which tables are needed (e.g., `cases`, `activities`, `variants`).

2. **Fetch Schemas with Tools**  
   For each relevant table, call the appropriate schema tool:  
   - Use `get_cases_schema()` for the `cases` table  
   - Use `get_activities_schema()` for the `activities` table  
   - Use `get_variants_schema()` for the `variants` table  

   **Do not proceed until all relevant schemas are fetched.**

3. **Format the Prompt for the SQL Agent**  
   After collecting all necessary schemas, generate a prompt that includes:
   - The name of each relevant table.
   - The schema details: structure and description of the fields.
   - Clear instructions on how to use the tables to answer the user's question.

4. **Restrictions for the Next Agent**
   - Instruct the next agent to use ONLY the tables provided in the schemas.
   - DO NOT include any SQL code yourself.
   - Emphasize that the next agent should only return the final SQL result, not reasoning or explanations.

5. **Output Format**  
   Return ONLY the formatted prompt. Do not include your own thoughts or reasoning chain in the output. *DO NOT INCLUDE THE QUERY ITSELF NEITHER*
"""
)

#messages = [HumanMessage(content="Identify late deliveries and the most common variant of the process.")]
#messages = prompt_agent.invoke({"messages": messages})
#for m in messages["messages"]:
 #   m.pretty_print()


In [4]:
@tool("execute_sql_with_prompt", description="Generates and runs a SQL query using the provided prompt and question")
def execute_sql_with_prompt(data: dict) -> str:
    print('data: ', data)
    question = data["question"]
    prompt = data["prompt"] 
    print("Prompt: ", prompt)
    general_instructions= """
    /no_think
    **IMPORTANT GUIDELINES**

    - Generate only one SQL Query.
    - The result must be executable as it is, so do not include any instructions, just the SQL code.
    - Only use the provided schemas to generate the SQL query, and do not reference any other tables or schemas.
    - You can perform JOINs between the tables, but you should not reference any other tables or schemas.
    """
    combined_prompt = general_instructions + prompt 
    result = run_sql_workflow(question, combined_prompt)
    return result

sql_generator_agent = create_react_agent(
    model=local_model,
    tools=[execute_sql_with_prompt],
    name="sql_generator_agent",
    prompt="""/no_think
        You are a SQL execution agent. Your task is to answer a high-level user question using SQL via tool calls. You must follow a step-by-step process to decompose the question, execute all required subqueries, and finally generate the answer based on those results.
        
        ### IMPORTANT
        - **ALWAYS USE `execute_sql_with_prompt` TO CALL SQL**
        - **DO NOT WRITE SQL YOURSELF**
        - **DO NOT ATTEMPT TO ANSWER THE QUESTION UNTIL ALL REQUIRED TOOL CALLS ARE COMPLETE**
        
        ## Step-by-Step Instructions
        
        1. **Analyze the Question**
           Determine whether the user's question can be answered with a single query or requires multiple steps. If it's complex, break it into smaller, manageable subquestions.
        
        2. **Decompose into Subquestions**
           If needed, divide the question into a sequence of subquestions that each represent one SQL task.
           - Each subquestion should focus on a specific aspect of the data needed.
           - Ensure that subquestions are ordered logically.
           - If a subquestion depends on data from a previous one, wait for the tool's result and use it in the next subquestion.
        
        3. **Call the SQL Tool for Each Subquestion**
           For every subquestion, use the `execute_sql_with_prompt` tool. Pass both the subquestion and the prompt exactly as received.
           - Wait for each tool call to complete.
           - Incorporate the results into the next step if needed.
        
        4. **Generate the Final Answer**
           Once all necessary subquestions have been answered and sufficient information is collected:
           - Use the results to formulate a complete and correct answer to the original user question.
           - DO NOT include any intermediate reasoning, SQL queries, or tool call details in the final response.
        
        ## Restrictions
        - DO NOT generate SQL yourself.
        - DO NOT return partial answers or summaries before all relevant subquestions have been resolved.
        - ONLY use the tool provided to get data.
        - ONLY return the final answer when confident all steps are complete.
        
        ## Output Format
        - Each subquestion should trigger a separate tool call.
        - Return only the final answer once all subquestions have been executed and their results synthesized.

        ### Tool Usage

        You MUST use the tool `execute_sql_with_prompt` to generate and execute SQL.
        
        - **This tool takes care of both generating AND running the SQL query.**
        - You MUST NOT generate SQL queries yourself.
        - Instead, you must always call the tool with the following format:
        
        {{
          "prompt": "<The full SQL prompt with schema and instructions>",
          "question": "<The specific subquestion this call is answering>"
        }}
        
        """
)

In [5]:
workflow = create_supervisor(
    agents=[prompt_agent, sql_generator_agent],
    model=local_model,
    tools=[
        create_custom_handoff_tool(
            agent_name="prompt_agent",
            name="handoff_to_prompt_agent",
            description="Transfer the natural language question to the SQL prompt formatting agent."
        ),
        create_custom_handoff_tool(
            agent_name="sql_generator_agent",
            name="handoff_to_sql_generator_agent",
            description="Transfer the question and formatted SQL prompt to the SQL generator agent for query execution."
        )
    ],
   prompt=(
        "/no_think\n"
        "You are a supervisor agent in charge of coordinating a multi-agent SQL and analysis workflow.\n"
        "Your job is to manage the following agents and tools to ensure the user’s question is answered step-by-step:\n\n"
    
        "## Agents:\n"
        "1. **prompt_agent** – Prepares detailed SQL prompts.\n"
        "   - Takes a **natural language question** and determines which tables are relevant.\n"
        "   - Uses schema tools to fetch the actual schema (never guesses it).\n"
        "   - Returns a formatted SQL prompt including schemas and instructions, NOT SQL code.\n\n"
    
        "2. **sql_generator_agent** – Handles SQL query execution.\n"
        "   - Receives the formatted prompt and a subquestion.\n"
        "   - Generates AND executes the SQL query using the `execute_sql_with_prompt` tool.\n"
        "   - Returns the result of the query.\n\n"
    
        "3. *(Optional future agents like an 'analysis_agent')* – Used to interpret results, perform aggregations, or answer questions that do not involve querying.\n\n"
    
        "## Tools:\n"
        "- **handoff_to_prompt_agent**: Use this to forward the original user question to the `prompt_agent` for schema resolution.\n"
        "- **handoff_to_sql_generator_agent**: Use this to send a subquestion and the SQL prompt to the `sql_generator_agent` for execution.\n\n"
    
        "## Step-by-Step Supervisor Logic:\n"
        "1. **Receive the user’s natural language question.**\n"
        "2. **Break it down into smaller subquestions** if it cannot be answered with a single SQL query.\n"
        "   - Each subquestion should be a specific queryable item.\n"
        "   - If later steps depend on earlier results, make that explicit in the subquestion.\n"
        "3. **Start by handing off the full question** to `prompt_agent` using `handoff_to_prompt_agent`.\n"
        "4. **Once you receive the formatted SQL prompt**, use it to answer subquestions by handing them off one-by-one to `sql_generator_agent` using `handoff_to_sql_generator_agent`.\n"
        "   - Always send data in this format: `{ \"prompt\": ..., \"question\": ... }`\n"
        "5. **Wait for results** from each subquestion before continuing.\n"
        "6. **Combine results** if needed and issue more subquestions, or perform analysis using another agent.\n\n"
    
        "## Restrictions:\n"
        "- NEVER write or execute SQL code yourself.\n"
        "- DO NOT fabricate schemas. Only use those returned by the prompt agent.\n"
        "- NEVER return final answers without going through the tool-based process.\n"
        "- You MUST use the appropriate handoff tool for each agent.\n\n"
    
        "You are responsible for routing, not computation. Follow this pattern strictly to ensure accuracy and modularity."
    ),
    output_mode="full_history"
)

In [ ]:
app = workflow.compile()
result = app.invoke({
    "messages": [
        {
            "role": "user",
            "content": "Identify late deliveries and the most common variant of the process."
        }
    ]
}
)

In [ ]:
for m in result["messages"]:
    m.pretty_print()

In [ ]:
sql= """
<think>

</think>

```sql
SELECT 
    c.delivery,
    c.estimated_delivery,
    v.id AS variant_id,
    v.number_cases
FROM 
    cases c
JOIN 
    variants v
ON 
    c.id = v.cases
ORDER BY 
    v.number_cases DESC
LIMIT 1;
"""
remove_think_blocks(sql)